In [ ]:
""" This file does the scimilarity annotation but as .ipnyb file it makes the debugging easy """

In [8]:
# Environment settings
import scanpy as sc
import warnings
from scimilarity.utils import lognorm_counts
from scimilarity import CellAnnotation, align_dataset
import utils_AT
import numpy as np
import pandas as pd
import csv

In [9]:
""""  ###### VARIABLES ########  """

# Select visualization options
ALL = 0
clstr = 0
uncnstrnd_prdctn = 0
cnstrnd_prdctn = 0
min_dist = 1
gene_score = 1
annotation_path = '../models/annotation_model_v1'
data_path = '../data/GSE136831_subsample.h5ad' #(AT)
""""  ###### VARIABLES ########  """

'"  ###### VARIABLES ########  '

In [10]:
ca = CellAnnotation(model_path=annotation_path)

In [6]:
# Model read, data read
sc.set_figure_params(dpi_save=800)
warnings.filterwarnings('ignore')
ca = CellAnnotation(model_path=annotation_path)
adams = sc.read(data_path)

#(AT) to match the target gene, all name of the gene needs to be uppercase
for indx_num in range(len(adams.var.index)):
     adams.var.index.values[indx_num] = adams.var.index[indx_num].upper()
#data processing
adams = align_dataset(adams, ca.gene_order)
adams = lognorm_counts(adams)
adams.obsm['X_scimilarity'] = ca.get_embeddings(adams.X)


In [4]:
""" UNCONSTRAINED ANNOTATION """

# - nn_idxs: indicies of cells in the SCimilarity reference.
# - nn_dists: the minimum distance within k=50 nearest neighbors.
# - nn_stats: a dataframe containing useful metrics
#           such as (distribution of celltypes in k=50 nearest neighbors.

# Processing
predictions, nn_idxs, nn_dists, nn_stats = ca.get_predictions_kNN(adams.obsm['X_scimilarity'])
adams.obs['predictions_unconstrained'] = predictions.values

""" UNCONSTRAINED ANNOTATION """

Get nearest neighbors finished in: 0.15995682875315348 min


100%|██████████| 50000/50000 [00:16<00:00, 3076.46it/s]


' UNCONSTRAINED ANNOTATION '

In [11]:
data_path = '../data/KPTracer-Data/expression/adata_processed_combined_SCANTN.h5ad'
adams = sc.read(data_path)

In [12]:
fm_basic_signature = ['SPP1', 'TREM2', 'GPNMB', 'MMP9', 'CHIT1', 'CHI3L1']
sc.tl.score_genes(adams, fm_basic_signature)
sig_query_threshold = adams.obs.score.quantile(.999)
cells_used_in_query = adams.obs.score>=sig_query_threshold
adams.obs['used_in_query'] = cells_used_in_query

In [13]:
from scimilarity.utils import get_centroid
avg_cell = get_centroid(adams.layers['counts'][adams.obs['used_in_query']])
avg_embedding = ca.get_embeddings(avg_cell)

In [14]:
from scimilarity import CellQuery

query_path = '../models/query_model_v1'
cq = CellQuery(model_path=annotation_path,
               cellsearch_path=query_path)

In [19]:
k = 10000
nn_idxs, nn_dists, results_metadata = cq.search(avg_embedding, k=k)

KeyboardInterrupt: 